## Recommendation Systems.
There are three main types of recommendation systems:
* **Content-based filtering** - looks at the item and recommends similar items eg action movie(Jumanji) then the recommendation can be a similar action movie (Tomb Raider)
* **Collaborative filtering** - looks at who liked what, then suggests these items.
* **Hybrid Models** - combines both for more nuanced recommendations.(Amazon, Netflix & Google Ads)

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd


In [9]:
items = ["movie id", "movie title", "release date", "video_release_date",
            "IMDb URL", "unknown", "Action", 'Adventure', "Animation",
              "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
              "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
              "Thriller", "War", "Western"]

movies = pd.read_csv('./data/mlk/u.item', sep="|", names=items, encoding='latin-1')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie id            1682 non-null   int64  
 1   movie title         1682 non-null   object 
 2   release date        1681 non-null   object 
 3   video_release_date  0 non-null      float64
 4   IMDb URL            1679 non-null   object 
 5   unknown             1682 non-null   int64  
 6   Action              1682 non-null   int64  
 7   Adventure           1682 non-null   int64  
 8   Animation           1682 non-null   int64  
 9   Children's          1682 non-null   int64  
 10  Comedy              1682 non-null   int64  
 11  Crime               1682 non-null   int64  
 12  Documentary         1682 non-null   int64  
 13  Drama               1682 non-null   int64  
 14  Fantasy             1682 non-null   int64  
 15  Film-Noir           1682 non-null   int64  
 16  Horror

In [10]:
movies = movies.drop(['video_release_date','IMDb URL'], axis=1)
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie id      1682 non-null   int64 
 1   movie title   1682 non-null   object
 2   release date  1681 non-null   object
 3   unknown       1682 non-null   int64 
 4   Action        1682 non-null   int64 
 5   Adventure     1682 non-null   int64 
 6   Animation     1682 non-null   int64 
 7   Children's    1682 non-null   int64 
 8   Comedy        1682 non-null   int64 
 9   Crime         1682 non-null   int64 
 10  Documentary   1682 non-null   int64 
 11  Drama         1682 non-null   int64 
 12  Fantasy       1682 non-null   int64 
 13  Film-Noir     1682 non-null   int64 
 14  Horror        1682 non-null   int64 
 15  Musical       1682 non-null   int64 
 16  Mystery       1682 non-null   int64 
 17  Romance       1682 non-null   int64 
 18  Sci-Fi        1682 non-null   int64 
 19  Thrill

Load in the user data.

In [12]:
ratings_columns = ["user id", "item id", "rating", "timestamp"]

ratings = pd.read_csv('./data/mlk/u.data', sep='\t', names=ratings_columns, encoding='latin-1')
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user id    100000 non-null  int64
 1   item id    100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [25]:
movies.head()

,movie id,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [13]:
ratings.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [29]:
# combined_data = pd.concat([movies, ratings], axis=1)

genres = ["unknown", "Action", 'Adventure', "Animation",
              "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
              "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
              "Thriller", "War", "Western"]

genre_features = movies[genres]



## Cosine Similarity
Instead of using angles directl, cosine similarity gives us a score between -1 and 1.
* 1 = items are identical in terms of direction
* 0 = they are completely different
* -1 = opposite direction

In [32]:
movies.head()

,movie id,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#get the matrix to use within our function.
cosine_matrix = cosine_similarity(genre_features)

#displaying similarity of first movie with others
cosine_matrix[0][:15]

array([1.        , 0.        , 0.        , 0.33333333, 0.        ,
       0.        , 0.        , 0.66666667, 0.        , 0.        ,
       0.        , 0.        , 0.57735027, 0.        , 0.        ])

In [34]:
movies.head(10)

,movie id,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Twelve Monkeys (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,8,Babe (1995),01-Jan-1995,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,9,Dead Man Walking (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Richard III (1995),22-Jan-1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
def movie_recommendation(movie_title, n=5):
    movie_index = movies[movies['movie title'] == movie_title].index[0]

    scores = list(enumerate(cosine_matrix[movie_index]))

    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    #return the scores ommitting the first movie and adding the 6th one to make five 
    return_scores = sorted_scores[1: n+1]
    #get movie indices and names
    movie_indices = [i[0] for i in return_scores]

    return movies['movie title'].iloc[movie_indices]



movie_recommendation("Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)")

8       Dead Man Walking (1995)
14    Mr. Holland's Opus (1995)
17    White Balloon, The (1995)
18        Antonia's Line (1995)
29         Belle de jour (1967)
Name: movie title, dtype: object

In [42]:
movies.tail(12)

,movie id,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1670,1671,"Further Gesture, A (1996)",20-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1671,1672,Kika (1993),01-Jan-1993,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1672,1673,Mirage (1995),01-Jan-1995,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1673,1674,Mamma Roma (1962),01-Jan-1962,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1674,1675,"Sunchaser, The (1996)",25-Oct-1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1675,1676,"War at Home, The (1996)",01-Jan-1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1676,1677,Sweet Nothing (1995),20-Sep-1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1677,1678,Mat' i syn (1997),06-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [48]:
def movie_recommendation(movie_title, n=5):
    movie_index = movies[movies['movie title'] == movie_title].index[0]

    scores = list(enumerate(cosine_matrix[movie_index]))

    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    #return the scores ommitting the first movie and adding the 6th one to make five 
    return_scores = sorted_scores[: n+1]
    #get movie indices and names
    movie_indices = [i[0] for i in return_scores]

    # return movies['movie title'].iloc[movie_indices]
    return return_scores
movie_recommendation('Dead Man Walking (1995)')

[(5, 1.0), (8, 1.0), (14, 1.0), (17, 1.0), (18, 1.0), (29, 1.0)]

In [44]:
movie_recommendation('B. Monkey (1998)')

885            Life Less Ordinary, A (1997)
1678                       B. Monkey (1998)
32                         Desperado (1995)
67                         Crow, The (1994)
89      So I Married an Axe Murderer (1993)
Name: movie title, dtype: object